In [ ]:
from draw import draw_attention

In [ ]:
import infos

In [ ]:
config = infos.config_16
target = 16
similar_wk = 1
model_path = infos.model_path_16

model = draw_attention(config=config, target=target, similar_wk=similar_wk, model_path=model_path, save_path='wk16.pdf')

In [ ]:
config = infos.config_17
target = 17
similar_wk = 6
model_path = infos.model_path_17

model = draw_attention(config=config, target=target, similar_wk=similar_wk, model_path=model_path, save_path='wk17.pdf')

In [ ]:
config = infos.config_18
target = 18
similar_wk = 14
model_path = infos.model_path_18

model = draw_attention(config=config, target=target, similar_wk=similar_wk, model_path=model_path, save_path='wk18.pdf')